<a href="https://colab.research.google.com/github/DavidBert/TP-MAPI3/blob/master/TP2_MAPI3_Sujet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification avec Keras

Nous allons dans ce TP réaliser des classifications d'images à l'aide de la librairie Keras.  
Keras est une API de réseaux de neurones de haut niveau. Elle joue le rôle d'interface avec des librairies de deep learnning telles que TensorFlow, CNTK ou Theano. Elle est aujourd'hui totalement intégrée dans les versions actuelles de TensorFlow et permet de prototyper et d'entrainer des réseaux de neurones de façon rapide et facile tout en béneficiant des fonctionnalités offertes par TensorFlow. 
![](https://lesdieuxducode.com/images/blog/titleimages/keras-tensorflow-logo.jpg)

# Multilayer-Perceptron

Pour commencer ce TP nous allons apprendre à utiliser Keras sur le même exemple que le TP précédent: la reconnaissance de chiffres manuscrits.  
Nous allons donc construire un premier réseau "fully-connected" qui prendra en entrée une image et retournera une classification
![](https://img.favpng.com/13/22/3/mnist-database-multilayer-perceptron-artificial-neural-network-statistical-classification-machine-learning-png-favpng-tDc3Ze2RegCutriyH12TfquqE.jpg)

Commençons par importer le datsset.  
Le dataset MNIST de classification de chiffres manuscrits est déjà intégré dans la libraries Keras.

In [0]:
import keras
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

Jetons un oeil au format du dataset:

In [0]:
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

## Question:
Affichez quelques éléments de X_train:

In [0]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
...

Avant de pouvoir entrainer notre futur réseau nous devons faire un peu de preprocessing sur nos données en les redimensionant et en les normalisant.  
Actuellement les données sont stockées dans un tenseur de shape (6000, 28, 28) de type uint8 avec des valeurs comprises entre [0, 255]. Nous devons les transformer en float32 dans un tenseur de taille (6000, 28*28) avec des valeurs comprises entre [0 et 1]

In [0]:
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

Les labels attendus par Keras dans des problèmes de classification doivent être forme de one-hot vectors.  
Par exemple le label 2 doit être représenter par le vecteur [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]. Nous devons convertir les labels $y$ sous cette forme avant de pouvoir les utiliser sous Keras.  
Keras possède une méthode permettant de faire cette convertion simplement:

In [0]:
print(f'Old version: {y_train[0]}')

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(f'One-hot encoded version: {y_train[0]}')

La façon la plus simple de construire un réseau sous Keras est d'utiliser les models "Sequential".  
Il suffit d'instancier un de ces models et d'y ajouter les couches dont on a besoin une par une.

In [0]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
# Nous rajoutons ici une première couche cachée de 512 neurones avec fonction d'acctivation ReLU
# Seul la première couche à besoin de savoir la taille de ses inputs
model.add(Dense(512, activation='relu', input_shape=(784,)))
# La dernière couche de notre réseau possède un appel à la fonction softmax.
# Cela permet de produire une distribution de probabilités sur les différentes calsses possible en sortie du réseau
model.add(Dense(num_classes, activation='softmax'))

# la methode summary permet de visualiser rapidement les informations du réseau
# regardez le nombre de paramètres utilisés
model.summary()

Keras possède aussi des outils permettant de visualiser l'architecture des réseaux

In [0]:
from keras.utils import plot_model
plot_model(model, to_file='mlp_model.png')

Avant de pouvoir entrainer le réseau nous devons configurer son processus d’apprentissage avec .compile() afin de lui fournir la fonction de perte qu'il cherchera à minimiser et l'optimiseur qu'il pourra utiliser pour cela.  
Enfin nous pouvons lui donner des metrics qui nous permettrons de l'évaluer.

In [0]:
from keras.optimizers import RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

Le code suivant permet de lancer un apprentissage de 20 epochs sur de batchs de taille 128 et de stocker les statistiques de l'apprentissage dans un objet history

In [0]:
batch_size = 128
epochs = 20

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, y_test))

La méthode évaluate permet de voir les résulats de l'apprentissage sur le jeu de test

In [0]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Nous pouvons afficher les courbes d'apprentissages grace aux statistiques conservées dans history

In [0]:
import matplotlib.pyplot as plt

def plot_learning_curves(history):
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

In [0]:
plot_learning_curves(history)

## Question:
Au vu des courbes nous avons clairement overfité sur nos données d'apprentissage.  
Vous avez vu en cours une méthode simple permettant de réduire le sur-apprentissage il s'agit du [dropout](http://www.jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf). Cette methode est très simple à mettre en oeuvre dans Keras. 
Construisez un nouveau model ayant la même architecture mais incluant du dropout `model.add(Dopout(rate))`et affichez les statistques de sont apprentissage. Que constatez vous?

In [0]:
from keras.layers import Dropout

model = Sequential()
...
model.summary()

model.compile(...)

history = model.fit(...)

plot_learning_curves(history)

Vous pouvez facilement enregistrer les poids de votre model:

In [0]:
import os

save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'MNIST_MLP_Keras.h5'

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Convolutional Neural Networks

![](https://miro.medium.com/max/395/1*1VJDP6qDY9-ExTuQVEOlVg.gif)

Nous allons maintenant implémenter les réseaux convolutionels que vous avez vu en cours. Ces réseaux sont capables de travailler directement avec des images en entrées.  
Important cependant, dans Keras les réseaux convolutionels prennent en entrées des tensors de la forme suivante (image_height, image_width, image_channels) (sans inclure la dimension du batch). Les images MNIST étant en noir et blanc elles ne possèdent qu'un seul canal. Nous allons donc redimensionner nos images pour qu'elles aient la taille suivante: (28, 28, 1)

In [0]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Keras attend un input de la forme [taille du batch, hauteur, largeur, nb_cannaux]
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Nous travaillons donc directement avec des images  28x28x1.  
Voici un exemple simple de réseau convolutionnel sous Keras.  
Ce réseau est constitué de plusieurs couches convolutionnelles `Conv2D`.
Chacune de ces couches possède des arguments, entre autres:
- `filters`: le nombre de filtres en sortie de la couche de convolution
- `kernel`: la taille des filtres de convolution
- `activation`: la fonction d'activation appliquée sur la couche
- `input_shape` la taille des entrées de la couche. Cet argument est uniquement nécéssaire sur la première couche les suivants calculeront automatiquement la taille de leurs entrée.  
La documentation complète de ces couches peut êêtre trouvée [ici](https://keras.io/layers/convolutional/).  
Le modèle possède aussi des couches `MaxPooling2D` réalisant les oppération de pooling une couche `Flatten` permettant d'applatir les matrices en sortie sous forme de vecteurs et des couches `Denses` que nous avons déjà utilisées.

In [0]:
from keras.layers import Conv2D, MaxPooling2D, Flatten

num_classes = 10

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

## Question
Comparez le nombre de paramètres du réseau avec le précédent

In [0]:
...

## Question:
Entrainez le réseaux sur X_train et affichez les courbes d'apprentissage

In [0]:
batch_size = 128
epochs = 12

...

plot_learning_curves(history)

## Question:
Affichez les scores sur X_test

In [0]:
print('Test loss:', ...)
print('Test accuracy:', ...)

# CIFAR

Nous allons cette fois-ci utiliser un CNN pour classifier des images en couleur.  
Nous allons utiliser le dataset CIFAR-10 constitué de 60 000 images, en couleur, de résolution 32*32.  
Le dataset est séparé en 2 parties, les données d’apprentissage (50 000 images) et les données de test (10 000 images).  
La tâche consiste à classifier les images du dataset parmis 10 classes.

In [0]:
from keras.datasets import cifar10
import numpy as np

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
num_train, img_channels, img_rows, img_cols =  X_train.shape
num_classes = len(np.unique(y_train))
print(X_train.shape)

Voici un exemple d'image de chaque classe

In [0]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']
fig = plt.figure(figsize=(10,5))
for i in range(num_classes):
    plt.subplot(2, 5, 1 + i)
    #get first exmple of ith class
    idx = np.where(y_train[:]==i)[0][0]
    image = X_train[idx,::]
    plt.imshow(image)
    plt.title(class_names[i])

## Question:
Similarement à ce ce que l'on a fait précédement avec le dataset MNIST, implémentez un resau CNN capable de catégoriser les images de CIFAR10.  
N'oubliez pas la préprocessing des données et des labels.  
Voici un exemple d'architecture mais n'hésitez pas à tester les votres.

* Conv2D: filter: 32, Kernel(3,3), padding='same', activation:relu  
* Conv2D: filter: 32, Kernel(3,3), padding='same', activation:relu
* MaxPooling2D: (2,2)
* Conv2D: filter: 64, Kernel(3,3), padding='same', activation:relu  
* Conv2D: filter: 64, Kernel(3,3), padding='same', activation:relu
* MaxPooling2D: (2,2)
* Flatten
* Dense 512 activation:ReLU
* Dense num_classes activation:softamx




## Questions
Entrainez votre réseau et affichez vos courbes d'apprentissage

In [0]:
batch_size = 64
num_classes = 10
epochs = 20

history = ...
...

# Tiny Imagenet

Nous allons maintenant nous entrainer sur des images de plus grandes résolutions.  
Pour cela nous utiliserons un sous-échantillon du célèbre dataset [Imagenet](http://www.image-net.org/challenges/LSVRC/2014/), et nous nous concentrerons sur 3 classes.  
Commençons par télécharger les données

In [0]:
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
!unzip imagenette2.tgz

In [0]:
!tar zxvf imagenette2.tgz

Afin de ne pas charger les données en RAM le code suivant utilise un générateur permettant de charger les images dans la RAM uniquement lorsque l'on en a besoin.  
Les images sont stockées dans le dossier imagenette2, lui mêême séparé en deux dossier train et val.  
Chacun d'eux contient 10 sous dossiers: un pour chaque classe.  
Il s'agit de l'arborescence attendue par les generateurs en Keras
Deux générateurs sont définis un pour l'apprentissage un autre pour la validation.  
Le préprocessing des images est géré par les générateurs regardez attentivement comment dans le code.

In [0]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = 'imagenette2'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Voici un exemple d'image de chaque classe

In [0]:
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img

plt.figure(figsize=(10,5))
plt.subplot(1,3,1)
img = load_img(os.path.join(train_dir, 'n01440764/ILSVRC2012_val_00000293.JPEG'))
plt.imshow(img)
plt.subplot(1,3,2)
img = load_img(os.path.join(train_dir, 'n02102040/n02102040_2474.JPEG'))
plt.imshow(img)
plt.subplot(1,3,3)
img = load_img(os.path.join(train_dir, 'n02979186/n02979186_14378.JPEG'))
plt.imshow(img)
plt.axis('off');

## Question:
Essayez d'apprendre sur ce jeu de données avec une architecture similaire à votre ancien modèle et affichez les courbes d'apprentissage.  
Pensez à modifier la taille des inputs de la première couche et le nombre de classes en sortie.  
Voici un exemple de code permettant de fiter votre modèle depuis un générateur:  
```python
history = model.fit_generator(
      train_generator,
      steps_per_epoch=128,
      epochs=20,
      validation_data=validation_generator,
      validation_steps=50)
```

In [0]:
...

history = ...

...

## Data Augmentation

Il est fort probable que vous ayez overfitté avec le modèle précédent.  
Nous allons voir dans cette section une des techniques souvent utilisées en computer vision pour améliorer la qualité des modèles et diminuer l'overfiting est la data-augmentation.   
L'idée est très simple: on applique des perturbations (rotations, zoom, ...) aux images déjà présentes dans le dataset afin de constituer de nouveaux échantillons d'apprentissage et ainsi robustifier notre modèle.  
Le code suivant définit un générateur qui génèrera à la volée les images perturbées.

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

augmenting_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40, # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2, # randomly shift images horizontally (fraction of total height)
    height_shift_range=0.2,# randomly shift images vertically (fraction of total height)
    shear_range=0.2,# set range for random shear
    zoom_range=0.2,# set range for random zoom
    horizontal_flip=True,# randomly flip images
    fill_mode='nearest' # set mode for filling points outside the input boundaries
)

Voici un exemple de data-augmentation de notre générateur

In [0]:
img = load_img(os.path.join(train_dir, 'n02102040/n02102040_2474.JPEG'))
X =np.array(img)
plt.figure(figsize=(11, 5))
flow = augmenting_datagen.flow(X[np.newaxis, :, :, :])
for i, X_augmented in zip(range(15), flow):
    plt.subplot(3, 5, i + 1)
    plt.imshow(X_augmented[0])
    plt.axis('off')

## Question:
Utilisez lancer un apprentissage avec le générateur faisant de la data-augmentation sur un modèle possédant la même architecture que le modèle précédent et affichez les courbes d'apprentissage.  
Le generateur servant pour la validation n'a pas besoin d'être augmenté.
Que remarquez vous?

### Attention la data-augmentation ralentit la vitesse d'apprentissage sour Keras traitez cette question en toute fin de scéance pour ne pas ralentir votre progréssion dans le notebook

In [0]:
model = ...

opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

train_generator = augmenting_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

history = ...

plot_learning_curves(history)

# Using a pre-trained convnet

Le réseau ne semble pas pouvoir apprendre rapidement sur le dataset.  
Nous allons cette fois-ci utiliser une stratégie différente:
nous allons récuperer un réseau pré-entrainé (en l'occurence sur Imagenet), couper les dérnières couches afin de récupérer les features qu'il utilise pour effectuer ses décisions. Nous utiliserons alors ces features pour entrainer un classifieur.  
Le code suivant récupère un réseau entrainé sur Imagenet et enlève les dernières couches du réseau.

In [0]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

Il s'agit d'un modèle VGG16, voici son architecture:
![](https://neurohive.io/wp-content/uploads/2018/11/vgg16-1-e1542731207177.png)
Nous récupérons dans notre cas les features en sortie de la dernière couche de pooling.  
Nos feature sont donc de dimension $7*7*512$

In [0]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
test_features, test_labels = extract_features(validation_dir, 1000)


## t-SNE

Il peut parfois être utilie de visualiser les features afin d'avoir une indication de leur qualité.  
Nous allons utiliser une technique de visualisation appelé t-SNE et dont voici la définition (source [Wikipedia](https://fr.wikipedia.org/wiki/Algorithme_t-SNE)):  

*L'algorithme t-SNE (t-distributed stochastic neighbor embedding) est une technique de réduction de dimension pour la visualisation de données développée par Geoffrey Hinton et Laurens van der Maaten. Il s'agit d'une méthode non-linéaire permettant de représenter un ensemble de points d'un espace à grande dimension dans un espace de deux ou trois dimensions, les données peuvent ensuite être visualisées avec un nuage de points. L'algorithme t-SNE tente de trouver une configuration optimale selon un critère de théorie de l'information pour respecter les proximités entre points : deux points qui sont proches (resp. éloignés) dans l'espace d'origine devront être proches (resp. éloignés) dans l'espace de faible dimension.*

*L'algorithme t-SNE se base sur une interprétation probabiliste des proximités. Une distribution de probabilité est définie sur les paires de points de l'espace d'origine de telle sorte que des points proches l'un de l'autre ont une forte probabilité d'être choisis tandis que des points éloignés ont une faible probabilité d'être sélectionnés. Une distribution de probabilité est également définie de la même manière pour l'espace de visualisation. L'algorithme t-SNE consiste à faire concorder les deux densités de probabilité, en minimisant la divergence de Kullback-Leibler entre les deux distributions par rapport à l'emplacement des points sur la carte.*

Parmis les bonne pratiques il est recommandé de procéder à une PCA en amont afin de ne pas travailler dans des espaces trop grands:  
*It is highly recommended to use another dimensionality reduction method (e.g. PCA for dense data or TruncatedSVD for sparse data) to reduce the number of dimensions to a reasonable amount (e.g. 50) if the number of features is very high. This will suppress some noise and speed up the computation of pairwise distances between samples. For more tips see Laurens van der Maaten’s FAQ.* (source [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html)

## Question
Visulalisez vos features à l'aide des methodes [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) et [t-SNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) de scikit-learn. 
Vous reduirez dans un premier temps la dimension de vos features grace à une PCA puis utiliserez la t-SNE.

In [0]:
train_features = np.reshape(train_features, (2000, 7 * 7 * 512))
test_features = np.reshape(test_features, (1000, 7 * 7 * 512))

In [0]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
X_pca = ...
X_tsne = ...

In [0]:
import seaborn as sns
plt.figure(figsize=(16,10))
sns.scatterplot(
    x=X_tsne[:,0], y=X_tsne[:,1],
    hue=train_labels,
    palette=sns.color_palette("hls", 10),
    legend="full",
)

## Random Forest

Nous pouvons utiliser n'importe quel classifieur pour classer nos images à partir de leur projection dans l'espace des features.
## Question  
Utilisez un [random forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) de scikit-learn pour classifier à partir des features et mesurez votre précision.

In [0]:
from sklearn.ensemble import RandomForestClassifier


In [0]:
from sklearn.metrics import accuracy_score
y_pred = ...
accuracy = accuracy_score(test_labels, y_pred)
print(f'accuracy:{accuracy}')

## New classifier

Entrainez un petit réseau (une couche cachée de 256 neurones et une couche de dropout) à partir des features et mesurez sa performance.

In [0]:
train_labels = keras.utils.to_categorical(train_labels, num_classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes)

In [0]:
model = Sequential()
model.add(
    ...

model.compile(optimizer=RMSprop(lr=2e-5),
              loss='categorical_crossentropy',
              metrics=['acc'])

...
plot_learning_curves(history)

Plutôt que de créer deux modèles, un pour le calcul des features déjà entrainé et un autre pour la prise de décision, nous pouvons définir un unique modèle composé du premier et auquel on ajoute une derniere couche de classification.

In [0]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation='relu', name='features'))
model.add(Dense(num_classes, activation='sigmoid'))

model.summary()

Nous allons geler les poids des couches calculant les features afin de ne pas tout ré-apprendre et d'accélérer l'entrainement.

In [0]:
print('Voici le nombre de couches "trainable":', len(model.trainable_weights))

In [0]:
conv_base.trainable = False

In [0]:
print('Voici le nombre de couches "trainable" après avoir gelé le model de base:', len(model.trainable_weights))

## Question:
Entrainez le modèle et affichez sa courbe d'apprentissage

In [0]:
...

## Fine Tunning
Il est possible d'ameliorer encore un peu la qualité des prédictions en effecuant ce que l'on appèle du fine tunning.  
Le principe est très simple, nous allons dégeler les dernières couches de notre modèle de base et les entrainer conjointement avec les couches que nous lui avons ajoutées. L'idée ici est d'ajuster un peu (on utilise un learning rate plus petit que précédement) les représentations abstraites des dernières couches calculant les features pour les rendre plus pertinentes pour notre problème.  
Dans notre cas nos features ont été apprises sur le même dataset Imagenet. Elles sont donc déjà suffisement pertinentes pour la tâche que l'on cherche à résoudre et l'on ne devrait pas forcément observer d'améliorations.  
Voici cependant un exemple de code permettant de réaliser du fine-tuning:

In [0]:
conv_base.summary()

In [0]:
conv_base.trainable = False

for layer in conv_base.layers:
    if layer.name in ['block5_conv1', 'block5_conv2', 'block5_conv3'] :
        layer.trainable = True

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=128,
      epochs=20,
      validation_data=validation_generator,
      validation_steps=50)

## Question:
Nous sommes ici en présence d'un overfitting essayez de le réduire.
Vous pouvez essayer cette fois-ci une autre méthode permettant de le reduire: la [batch normalisation](https://keras.io/layers/normalization/)


In [0]:
...